# First, load the data

In [6]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import os
import napari
import numpy as np
from DLC_for_WBFM.utils.projects.finished_project_data import ProjectData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
folder_name = "/scratch/neurobiology/zimmer/Charles/dlc_stacks/students"

fname = "bright_worm7"
project_data = ProjectData.load_final_project_data_from_config(os.path.join(folder_name, fname))

ConfigFile  : INFO     Set up logger with name: ConfigFile


File for mode automatic exists at precendence: 1/3
Read data from: /scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/3-tracking/postprocessing/combined_3d_tracks.h5
ConfigFile  : INFO     Starting threads to read data...


ConfigFile  : INFO     Read all data from files: ['/scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/3-tracking/postprocessing/combined_3d_tracks.h5', '/scratch/neurobiology/zimmer/ulises/wbfm/20211217/data/worm7/2021-12-17_17-38-56_worm7-channel-0-pco_camera1/2021-12-17_17-38-56_worm7-channel-0-pco_camera1bigtiff.zarr.zip', '/scratch/neurobiology/zimmer/ulises/wbfm/20211217/data/worm7/2021-12-17_17-38-56_worm7-channel-1-pco_camera2/2021-12-17_17-38-56_worm7-channel-1-pco_camera2bigtiff.zarr.zip', '/scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/4-traces/red_traces.h5', '/scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/4-traces/green_traces.h5', '/scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/2-training_data/training_data_tracks.pickle', '/scratch/neurobiology/zimmer/Charles/dlc_stacks/students/bright_worm7/2-training_data/reindexed_masks.zarr', '/scratch/neurobiology/zimmer/Charles/dlc_stacks/student

# Second, use napari to look at basic data

## Napari is organized in layers; start with one

In [22]:
red_data = project_data.red_data
red_data

<zarr.core.Array (1550, 21, 650, 900) uint8>

In [23]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(red_data, colormap='red')

/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/napari/plugins/_plugin_manager.py:511: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Image layer 'red_data' at 0x7fbf8daae690>

## Add another layer to an existing viewer

In [24]:
green_data = project_data.green_data
green_data

<zarr.core.Array (1550, 21, 650, 900) uint8>

In [25]:
viewer.add_image(green_data, colormap='green', opacity=0.5)

<Image layer 'green_data' at 0x7fbf7cb46d90>

## Add a different type of layer: segmentation

In [28]:
seg_data = project_data.raw_segmentation
seg_data

<zarr.core.Array (1550, 21, 650, 900) uint16>

In [27]:
viewer.add_labels(seg_data)

<Labels layer 'seg_data' at 0x7fbf7cba7a50>

## Napari has many additional layers as well:

https://napari.org/howtos/layers/index.html

# Third, add interactivity to napari

Using this library: https://napari.org/magicgui/examples/napari/napari_img_math.html

In [29]:
from magicgui import magicgui

## Add a changing layer

In [31]:
@magicgui(call_button="Change time point")
def display_target_volume(viewer: napari.Viewer, t=34):
    """Updates the data of a specific layer"""
    target_volume = project_data.red_data[t]
    viewer.layers['Target Volume'].data = target_volume
    
# create a viewer and add the data
viewer = napari.Viewer(ndisplay=3)

red_data = project_data.red_data
viewer.add_image(red_data, colormap='red')

# Initialize the changing layer
viewer.add_image(red_data[34], name="Target Volume", colormap="PiYG", opacity=0.3)

# add our new magicgui widget to the viewer
viewer.window.add_dock_widget(display_target_volume)

/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/napari/plugins/_plugin_manager.py:511: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


## Calculate something using a user-defined layer

In [43]:
@magicgui(call_button="Sum ROI")
def sum_under_roi(viewer: napari.Viewer):
    """Sums values beneath a user-added layer"""
    mask_layer = viewer.layers['Shapes']
    masks = mask_layer.to_labels(red_data[0,0,...].shape)
    # print(masks)
    # dat = 
    # Get current values
    t, z, x, y = viewer.dims.current_step
    # Get just this slice
    current_dat = viewer.layers['red_data'].data[t, z, ...]
    
    print(current_dat.shape)
    print(masks.shape)
    summed_roi = np.sum(current_dat[masks==1])
    
    print(f"Value under the ROI: {summed_roi}")

# add our new magicgui widget to the viewer
viewer.window.add_dock_widget(sum_under_roi)

(650, 900)
(650, 900)
Value under the ROI: 9404
(650, 900)
(650, 900)
Value under the ROI: 9404
(650, 900)
(650, 900)
Value under the ROI: 9404
(650, 900)
(650, 900)
Value under the ROI: 14642
(650, 900)
(650, 900)
Value under the ROI: 253043


In [39]:
viewer.dims.current_step

(33, 10, 325, 450)